<a href="https://colab.research.google.com/github/MASAYA-KINOSHITA/first-Repository/blob/main/Nishika_%E3%82%BD%E3%83%95%E3%83%88%E3%82%A6%E3%82%A7%E3%82%A2%E7%95%B0%E5%B8%B8_%E3%83%81%E3%83%A5%E3%83%BC%E3%83%88%E3%83%AA%E3%82%A2%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

pd.set_option('max_colwidth', 500)
pd.set_option('max_columns', 500)
pd.set_option('max_rows', 500)

%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import matplotlib

import json, os, gc, math, time
import datetime
import collections
from tqdm import tqdm
import glob

from statistics import mean
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import KFold, GroupKFold, StratifiedKFold

from sklearn import metrics

import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore")

In [2]:
#メモリ節約用の関数
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
class Config():
    NAME = 'sample_for_beginner'
    INPUT_PATH = '/content/drive/MyDrive/Nishika ソフトウェアの異常検知/'
    OUTPUT_PATH = '/content/drive/MyDrive/Nishika ソフトウェアの異常検知/output/'
    SUBMIT_PATH = '/content/drive/MyDrive/Nishika ソフトウェアの異常検知/提出用ファイル/'
    TARGET = 'Anomaly' # this is fare
    NUM_FOLD = 5
    SEED_FOLD = 71
    SEED_MODEL = 71
    
CFG = Config()

In [4]:
train = pd.read_csv(os.path.join(CFG.INPUT_PATH, 'train.zip (Unzipped Files)/train.csv'))
test = pd.read_csv(os.path.join(CFG.INPUT_PATH, 'test.zip (Unzipped Files)/test.csv'))
sample_submission = pd.read_csv(os.path.join(CFG.INPUT_PATH, 'data.zip (Unzipped Files)/sample_submission.csv'))

In [5]:
reduce_mem_usage(train, verbose=True)
reduce_mem_usage(test, verbose=True)

Mem. usage decreased to 628.04 Mb (72.4% reduction)
Mem. usage decreased to 1077.57 Mb (71.4% reduction)


,id,timestamp,host,process,"Active connections : (MXBean(com.bea:Name=source01,Type=JDBCConnectionPoolRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source01,Type=JDBCDataSourceRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source02,Type=JDBCConnectionPoolRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source02,Type=JDBCDataSourceRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source03,Type=JDBCConnectionPoolRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source03,Type=JDBCDataSourceRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source04,Type=JDBCConnectionPoolRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source04,Type=JDBCDataSourceRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source05,Type=JDBCConnectionPoolRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source05,Type=JDBCDataSourceRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source06,Type=JDBCConnectionPoolRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source06,Type=JDBCDataSourceRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source08,Type=JDBCConnectionPoolRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source08,Type=JDBCDataSourceRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source09,Type=JDBCConnectionPoolRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source09,Type=JDBCDataSourceRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source10,Type=JDBCConnectionPoolRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source10,Type=JDBCDataSourceRuntime).ActiveConnectionsCurrentCount)","Active transactions : (MXBean(com.bea:Name=JTARuntime,Type=JTARuntime).ActiveTransactionsTotalCount)","Available db connection activity : (d/dx (MXBean(com.bea:Name=source01,Type=JDBCConnectionPoolRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source01,Type=JDBCDataSourceRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source02,Type=JDBCConnectionPoolRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source02,Type=JDBCDataSourceRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source03,Type=JDBCConnectionPoolRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source03,Type=JDBCDataSourceRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source04,Type=JDBCConnectionPoolRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source04,Type=JDBCDataSourceRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source05,Type=JDBCConnectionPoolRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source05,Type=JDBCDataSourceRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source06,Type=JDBCConnectionPoolRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source06,Type=JDBCDataSourceRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source08,Type=JDBCConnectionPoolRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source08,Type=JDBCDataSourceRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source09,Type=JDBCConnectionPoolRuntime).NumAvailable))","Available db connection activity : (d/dx (MXB

In [6]:
train.head()

,timestamp,host,Anomaly,process,"Active connections : (MXBean(com.bea:Name=source01,Type=JDBCConnectionPoolRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source01,Type=JDBCDataSourceRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source02,Type=JDBCConnectionPoolRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source02,Type=JDBCDataSourceRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source03,Type=JDBCConnectionPoolRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source03,Type=JDBCDataSourceRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source04,Type=JDBCConnectionPoolRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source04,Type=JDBCDataSourceRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source05,Type=JDBCConnectionPoolRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source05,Type=JDBCDataSourceRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source06,Type=JDBCConnectionPoolRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source06,Type=JDBCDataSourceRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source08,Type=JDBCConnectionPoolRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source08,Type=JDBCDataSourceRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source09,Type=JDBCConnectionPoolRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source09,Type=JDBCDataSourceRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source10,Type=JDBCConnectionPoolRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source10,Type=JDBCDataSourceRuntime).ActiveConnectionsCurrentCount)","Active transactions : (MXBean(com.bea:Name=JTARuntime,Type=JTARuntime).ActiveTransactionsTotalCount)","Available db connection activity : (d/dx (MXBean(com.bea:Name=source01,Type=JDBCConnectionPoolRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source01,Type=JDBCDataSourceRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source02,Type=JDBCConnectionPoolRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source02,Type=JDBCDataSourceRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source03,Type=JDBCConnectionPoolRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source03,Type=JDBCDataSourceRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source04,Type=JDBCConnectionPoolRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source04,Type=JDBCDataSourceRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source05,Type=JDBCConnectionPoolRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source05,Type=JDBCDataSourceRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source06,Type=JDBCConnectionPoolRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source06,Type=JDBCDataSourceRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source08,Type=JDBCConnectionPoolRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source08,Type=JDBCDataSourceRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source09,Type=JDBCConnectionPoolRuntime).NumAvailable))","Available db connection activity : (d/dx

### EDA

### 特徴量エンジニアリング
今回は基本的なカテゴリーエンコーディングにラベルエンコーディングを使用する。


日付についても単純に年月週等を取り出す。

In [7]:
def get_label_encoding(df , cols):
    """label_encoding
    """
    for col in cols:
        df[col].fillna("missing", inplace=True)
        le = LabelEncoder()
        le = le.fit(df[col])
        df["LE=" + col] = le.transform(df[col])
            
    return df
    
def get_count_encoding(df, cols):
    """count_encoding
    """
    for col in cols:
        counter = collections.Counter(df[col].values)
        count_dict = dict(counter.most_common())
        encoded = df[col].map(lambda x: count_dict[x]).values
        df["CE=" + col] = encoded
        
    return df

In [8]:
def get_date_feature(_df, col):
    """date to feature
    """

    date_siries = pd.to_datetime(_df[col])
    _df[col + "_year"] = date_siries.dt.year
    _df[col + "_month"] = date_siries.dt.month 
    _df[col + "_day"] = date_siries.dt.day 
    _df[col + "_week"] = date_siries.dt.dayofweek
    _df[col + "_hour"] = date_siries.dt.hour
    _df[col + "_minute"] = date_siries.dt.minute 
    _df[col + "_yymmdd"] = date_siries.dt.strftime('%Y%m%d').astype(np.int32) 

    return _df

特徴量生成

In [9]:
    #データが多いので一度testに統合する。
    # 訓練データとテストデータの結合
    test = pd.concat([train,test ],axis=0).reset_index(drop=True)
    
    # 日付データの処理
    #_df["timestamp"] = _df["timestamp"].fillna(0)
    test = get_date_feature(test, "timestamp")
    # ラベルエンコーディング
    cat_cols = ["host", "process"]
    test = get_label_encoding(test, cat_cols)
    # 不要なカラムの削除
    test= test.drop(["timestamp", "host", "process"], axis=1)
    # 再度訓練データとテストデータを分割
    train = test.iloc[:train.shape[0], :]
    test = test.iloc[train.shape[0]:, :]
    
    # 目的変数を取り出して返す
    target = train[CFG.TARGET]
    train = train.drop(CFG.TARGET, axis=1)
    test = test.drop(CFG.TARGET, axis=1)

In [10]:
train.head()

,"Active connections : (MXBean(com.bea:Name=source01,Type=JDBCConnectionPoolRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source01,Type=JDBCDataSourceRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source02,Type=JDBCConnectionPoolRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source02,Type=JDBCDataSourceRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source03,Type=JDBCConnectionPoolRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source03,Type=JDBCDataSourceRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source04,Type=JDBCConnectionPoolRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source04,Type=JDBCDataSourceRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source05,Type=JDBCConnectionPoolRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source05,Type=JDBCDataSourceRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source06,Type=JDBCConnectionPoolRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source06,Type=JDBCDataSourceRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source08,Type=JDBCConnectionPoolRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source08,Type=JDBCDataSourceRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source09,Type=JDBCConnectionPoolRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source09,Type=JDBCDataSourceRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source10,Type=JDBCConnectionPoolRuntime).ActiveConnectionsCurrentCount)","Active connections : (MXBean(com.bea:Name=source10,Type=JDBCDataSourceRuntime).ActiveConnectionsCurrentCount)","Active transactions : (MXBean(com.bea:Name=JTARuntime,Type=JTARuntime).ActiveTransactionsTotalCount)","Available db connection activity : (d/dx (MXBean(com.bea:Name=source01,Type=JDBCConnectionPoolRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source01,Type=JDBCDataSourceRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source02,Type=JDBCConnectionPoolRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source02,Type=JDBCDataSourceRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source03,Type=JDBCConnectionPoolRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source03,Type=JDBCDataSourceRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source04,Type=JDBCConnectionPoolRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source04,Type=JDBCDataSourceRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source05,Type=JDBCConnectionPoolRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source05,Type=JDBCDataSourceRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source06,Type=JDBCConnectionPoolRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source06,Type=JDBCDataSourceRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source08,Type=JDBCConnectionPoolRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source08,Type=JDBCDataSourceRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source09,Type=JDBCConnectionPoolRuntime).NumAvailable))","Available db connection activity : (d/dx (MXBean(com.bea:Name=source09,

## モデリング

In [11]:
def fit_lgbm(train, test, y, groups=None, params: dict=None, n_splits=5, verbose=100, early_stopping_rounds=100):
    """train lightgbm
    """
    
    models = []
    scores = []
    iterations = []
    oof_preds = np.zeros((train.shape[0],))
    sub_preds = np.zeros((test.shape[0],))
    
    folds = StratifiedKFold(n_splits=n_splits, random_state=CFG.SEED_FOLD, shuffle=True)
    for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train, target)):
        
        print("Fold is :", n_fold+1)
        trn_x, trn_y = train.iloc[trn_idx], y.iloc[trn_idx]
        val_x, val_y = train.iloc[val_idx], y.iloc[val_idx]
        trn_x = trn_x.values
        val_x = val_x.values
        
        clf = lgb.LGBMClassifier(**params)
        
        clf.fit(trn_x, trn_y, 
                eval_set= [(trn_x, trn_y), (val_x, val_y)], 
                verbose=verbose, early_stopping_rounds=early_stopping_rounds
               )
    
        oof_preds[val_idx] = clf.predict(val_x, num_iteration=clf.best_iteration_)
        sub_preds += clf.predict(test, num_iteration=clf.best_iteration_) / n_splits
        
        gc.collect()
        
        oof_preds = np.clip(oof_preds, 0, np.inf)
        sub_preds = np.clip(sub_preds, 0, np.inf)
        score = np.sqrt(metrics.mean_squared_error(y[val_idx], oof_preds[val_idx]))

        
        print("CV:{} RMSLE:{}".format(n_fold+1,score))
        
        iterations.append(clf.best_iteration_)
        scores.append(score)
        models.append(clf)
    
    return oof_preds, sub_preds, models, scores

In [12]:
params = {
    'objective': 'binary',
    'metric':'binary_logloss',
    'boosting_type': 'gbdt',
    'learning_rate': 0.1,
    'max_depth': -1,
    'num_leaves': 31,
    'n_estimators': 100000,
    "importance_type": "gain",
    'random_state': CFG.SEED_MODEL,
}

In [13]:
oof_pred, sub_pred, models, fold_scores = fit_lgbm(train, test, target,
                                                   params=params,
                                                   n_splits=5,
                                                   early_stopping_rounds=100)

Fold is : 1
Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 4.38286e-06	valid_1's binary_logloss: 7.94269e-05
[200]	training's binary_logloss: 1.44471e-08	valid_1's binary_logloss: 7.94782e-05
Early stopping, best iteration is:
[127]	training's binary_logloss: 6.68684e-07	valid_1's binary_logloss: 7.34021e-05
CV:1 RMSLE:0.0033657015456788437
Fold is : 2
Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 4.58e-06	valid_1's binary_logloss: 0.000113195
[200]	training's binary_logloss: 1.37967e-08	valid_1's binary_logloss: 0.000135244
Early stopping, best iteration is:
[116]	training's binary_logloss: 1.44292e-06	valid_1's binary_logloss: 0.000107582
CV:2 RMSLE:0.004759827512688043
Fold is : 3
Training until validation scores don't improve for 100 rounds.
[100]	training's binary_logloss: 0.00153079	valid_1's binary_logloss: 0.00487434
Early stopping, best iteration is:
[44]	training's binary_loglos

In [14]:
fold_scores

[0.0033657015456788437,
 0.004759827512688043,
 0.006938581313561285,
 0.003365706311500013,
 0.0037629740524070554]

In [15]:
mean(fold_scores)

0.004438558147167048

## 特徴量重要度

In [16]:
import matplotlib
dt_now = datetime.datetime.now()
today = dt_now.strftime('%Y-%m-%d %H-%M')
feature_importance_df = pd.DataFrame()
for i, model in enumerate(models):
    _df = pd.DataFrame()
    _df['feature_importance'] = model.feature_importances_
    _df['column'] = test.columns
    _df['fold'] = i + 1
    feature_importance_df = pd.concat([feature_importance_df, _df], axis=0, ignore_index=True)

order = feature_importance_df.groupby('column')\
    .sum()[['feature_importance']]\
    .sort_values('feature_importance', ascending=False).index[:100]

fig, ax = plt.subplots(figsize=(120, 120))
plt.title(CFG.TARGET, fontsize=24)
sns.set_theme(style="whitegrid")
sns.barplot(data=feature_importance_df, x='feature_importance', y='column', order=order, palette='husl')
ax.tick_params(axis='x', rotation=90)
fig.tight_layout()
plt.savefig(os.path.join(CFG.OUTPUT_PATH, CFG.NAME+"_feature_importance.png"))

In [20]:
#重要度のカラムを表示
order

Index(['GC activity : (incld/dx (MXBean(java.lang:name=PS Scavenge,type=GarbageCollector).CollectionCount))',
       'Stuck threads : (MXBean(com.bea:Name=ThreadPoolRuntime,Type=ThreadPoolRuntime).StuckThreadCount)',
       'LE=host',
       'Memory space usage : ((MXBean(java.lang:name=Code Cache,type=MemoryPool).Usage.committed / MXBean(java.lang:name=Code Cache,type=MemoryPool).Usage.max))',
       'Stuck threads : (MXBean(com.bea:ApplicationRuntime=sys01,Name=default,Type=WorkManagerRuntime).StuckThreadCount)',
       'Connection delay : (MXBean(com.bea:Name=source06,Type=JDBCConnectionPoolRuntime).ConnectionDelayTime)',
       'Daemon thread count : (MXBean(java.lang:type=Threading).DaemonThreadCount)',
       'Connection delay : (MXBean(com.bea:Name=source09,Type=JDBCConnectionPoolRuntime).ConnectionDelayTime)',
       'Connection delay : (MXBean(com.bea:Name=source05,Type=JDBCConnectionPoolRuntime).ConnectionDelayTime)',
       'timestamp_week',
       'Last GC duration : (MXBea

## サブミットファイルの作成

In [21]:
sample_submission["Anomaly"] =sub_pred
sample_submission.to_csv(os.path.join(CFG.SUBMIT_PATH, CFG.NAME+".csv"), index=False)